<a href="https://colab.research.google.com/github/theantigone/ngram-java-ai/blob/main/NGRAM_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pygments.lexers.jvm import JavaLexer
from pygments.token import Token
from collections import defaultdict, Counter
import math
import json
import random
import re
import pickle

def tokenize_java_code(code):
    """Tokenizes Java code using Pygments."""
    lexer = JavaLexer()
    return [t[1] for t in lexer.get_tokens(code) if t[0] not in Token.Text]


In [ ]:
def build_ngram_model(corpus, n):
    """Builds an N-gram model from tokenized Java methods."""
    ngram_counts = defaultdict(int)
    context_counts = defaultdict(int)

    for method in corpus:
        tokens = tokenize_java_code(method)
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            context = tuple(tokens[i:i+n-1])
            ngram_counts[ngram] += 1
            context_counts[context] += 1

    print(f"Number of N-grams: {len(ngram_counts)}")
    return ngram_counts, context_counts


In [ ]:
def compute_probabilities(ngram_counts, context_counts):
    """Computes conditional probabilities for N-grams."""
    probabilities = {}
    for ngram, count in ngram_counts.items():
        context = ngram[:-1]
        probabilities[ngram] = count / context_counts[context]
    return probabilities


In [ ]:
def perplexity(test_corpus, probabilities, n):
    """Computes perplexity for the N-gram model on a test set."""
    log_prob_sum = 0
    N = 0

    for method in test_corpus:
        tokens = tokenize_java_code(method)
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            prob = probabilities.get(ngram, 1e-6)  # Smoothing for unseen cases
            log_prob_sum += math.log2(prob)
            N += 1

    return 2 ** (-log_prob_sum / N)


In [ ]:
def iterative_prediction_x(probabilities, context, n):
    """Predicts the next token iteratively given a starting context."""
    predictions = []

    for _ in range(10):  # Limit predictions to avoid infinite loops
        candidates = {ngram[-1]: prob for ngram, prob in probabilities.items() if ngram[:-1] == context}

        if not candidates:
            break  # Stop if no valid prediction

        next_token = max(candidates, key=candidates.get)  # Most probable token
        predictions.append((next_token, round(candidates[next_token], 3)))
        context = context[1:] + (next_token,)  # Update context

    return predictions


In [ ]:
def generate_results_json(test_corpus, probabilities, n, filename):
    """Generates a JSON file with token predictions and probabilities."""
    results = {}

    for i, method in enumerate(test_corpus[:100]):  # Limit to 100 examples
        tokens = tokenize_java_code(method)[:n-1]
        context = tuple(tokens)
        results[str(i)] = iterative_prediction_x(probabilities, context, n)

    with open(filename, "w") as f:
        json.dump(results, f, indent=4)

    print(f"Saved predictions to {filename}")
    # Brief example
    if results:
        first_method_predictions = results["0"]
        if first_method_predictions:
            print(f"First prediction for method 0: {first_method_predictions[0]}")


In [ ]:
def generate_vocabulary(corpus):
    """Generates a vocabulary from the given corpus (training + eval + test)."""
    vocab = set()
    for method in corpus:
        tokens = tokenize_java_code(method)
        vocab.update(tokens)
    return vocab

In [ ]:
def load_methods_from_file(filename):
    """
    Reads a file containing methods enclosed in quotes and extracts each method as a single string.
    Uses a regular expression with DOTALL to capture methods spanning multiple lines.
    """
    with open(filename, "r") as f:
        content = f.read()
    # Extract text between double quotes (non-greedy) over multiple lines.
    methods = re.findall(r'"(.*?)"', content, re.DOTALL)
    # Strip whitespace from each method and filter out any empty strings.
    methods = [method.strip() for method in methods if method.strip()]
    return methods

In [ ]:
# def main():
#     # Load dataset from cleaned corpus (e.g., 7561 methods)
#     with open("/content/drive/Shareddrives/CSCI_420/data/new.txt", "r") as f:
#         corpus = [line.strip() for line in f]

#     print(f"Total methods in corpus: {len(corpus)}")  # Expected: 7561 methods

#     # Vocabulary Generation (using training + eval + test sets)
#     vocab = generate_vocabulary(corpus)
#     print(vocab)
#     print(f"Vocabulary size: {len(vocab)}")  # ZZZ number of code tokens

#     # Shuffle and split dataset: 80% training, 10% eval, 10% test
#     random.shuffle(corpus)
#     train_set = corpus[:int(0.8 * len(corpus))]
#     eval_set = corpus[int(0.8 * len(corpus)):int(0.9 * len(corpus))]
#     test_set = corpus[int(0.9 * len(corpus)):]

#     print(f"Train set: {len(train_set)} methods")
#     print(f"Eval set: {len(eval_set)} methods")
#     print(f"Test set: {len(test_set)} methods")

#     # Model Training & Evaluation on Student's Data
#     # Experiment with n = 3, n = 5, and n = 9; select best based on perplexity
#     best_n = None
#     best_perplexity = float("inf")
#     results = {}

#     for n in [3, 5, 9]:
#         print(f"Evaluating {n}-gram model on training data")
#         ngram_counts, context_counts = build_ngram_model(train_set, n)
#         probabilities = compute_probabilities(ngram_counts, context_counts)

#         pp = perplexity(eval_set, probabilities, n)
#         results[n] = pp
#         print(f"{n}-gram model Perplexity on Eval set: {pp}")

#         if pp < best_perplexity:
#             best_perplexity = pp
#             best_n = n

#     print(f"Selected best N: {best_n} (Perplexity: {best_perplexity})")
#     # According to our report, n = 3 was selected (with perplexity ~91.297)

#     # Train best model on full training set
#     best_ngram_counts, best_context_counts = build_ngram_model(train_set, best_n)
#     best_probabilities = compute_probabilities(best_ngram_counts, best_context_counts)

#     # Compute perplexity on the full test set for student model
#     test_pp = perplexity(test_set, best_probabilities, best_n)
#     print(f"Test set Perplexity for {best_n}-gram model: {test_pp}")  # Replace YYY.00000 with actual value

#     # Generate JSON output for student model (first 100 predictions)
#     generate_results_json(test_set, best_probabilities, best_n, "results_student_model.json")

#     # Training, Evaluation, and Testing on the Instructor-Provided Corpus
#     with open("/content/drive/Shareddrives/CSCI_420/data/training.txt", "r") as f:
#         instructor_corpus = [line.strip() for line in f]

#     print(f"Total methods in instructor's corpus: {len(instructor_corpus)}")
#     random.shuffle(instructor_corpus)
#     instructor_train_set = instructor_corpus[:int(0.8 * len(instructor_corpus))]
#     instructor_eval_set = instructor_corpus[int(0.8 * len(instructor_corpus)):int(0.9 * len(instructor_corpus))]

#     best_instructor_n = None
#     best_instructor_perplexity = float("inf")

#     for n in [3, 5, 9]:
#         print(f"Evaluating {n}-gram model on instructor's training data")
#         ngram_counts, context_counts = build_ngram_model(instructor_train_set, n)
#         probabilities = compute_probabilities(ngram_counts, context_counts)

#         pp = perplexity(instructor_eval_set, probabilities, n)
#         print(f"{n}-gram model Perplexity on Instructor's Eval set: {pp}")
#         if pp < best_instructor_perplexity:
#             best_instructor_perplexity = pp
#             best_instructor_n = n

#     print(f"Best N for instructor model: {best_instructor_n} (Perplexity: {best_instructor_perplexity})")
#     # According to our report, n = 3 was selected for the instructor corpus as well

#     # Train best instructor model
#     best_instructor_ngram_counts, best_instructor_context_counts = build_ngram_model(instructor_train_set, best_instructor_n)
#     best_instructor_probabilities = compute_probabilities(best_instructor_ngram_counts, best_instructor_context_counts)

#     # Compute perplexity on test set for instructor model
#     instructor_test_pp = perplexity(test_set, best_instructor_probabilities, best_instructor_n)
#     print(f"Test set Perplexity for instructor's {best_instructor_n}-gram model: {instructor_test_pp}")

#     # Generate JSON output for instructor model (first 100 predictions)
#     generate_results_json(test_set, best_instructor_probabilities, best_instructor_n, "results_teacher_model.json")

# if __name__ == "__main__":
#     main()

In [ ]:
def student_train():
    # Load dataset from cleaned corpus (e.g., 7561 methods)
    with open("/content/drive/Shareddrives/CSCI_420/data/new.txt", "r") as f:
        corpus = [line.strip() for line in f]

    print(f"Total methods in corpus: {len(corpus)}")  # Expected: 7561 methods

    # Vocabulary Generation (using training + eval + test sets)
    vocab = generate_vocabulary(corpus)
    # print(vocab)
    print(f"Vocabulary size: {len(vocab)}")  # ZZZ number of code tokens

    # Shuffle and split dataset: 80% training, 10% eval, 10% test
    random.shuffle(corpus)
    train_set = corpus[:int(0.8 * len(corpus))]
    eval_set = corpus[int(0.8 * len(corpus)):int(0.9 * len(corpus))]
    test_set = corpus[int(0.9 * len(corpus)):]

    print(f"Train set: {len(train_set)} methods")
    print(f"Eval set: {len(eval_set)} methods")
    print(f"Test set: {len(test_set)} methods")

    # Model Training & Evaluation on Student's Data
    # Experiment with n = 3, n = 5, and n = 9; select best based on perplexity
    best_n = None
    best_perplexity = float("inf")
    results = {}

    for n in [3, 5, 9]:
        print(f"Evaluating {n}-gram model on training data")
        ngram_counts, context_counts = build_ngram_model(train_set, n)
        probabilities = compute_probabilities(ngram_counts, context_counts)

        pp = perplexity(eval_set, probabilities, n)
        results[n] = pp
        print(f"{n}-gram model Perplexity on Eval set: {pp}")

        if pp < best_perplexity:
            best_perplexity = pp
            best_n = n

    print(f"Selected best N: {best_n} (Perplexity: {best_perplexity})")
    # According to our report, n = 3 was selected (with perplexity ~91.297)

    # Train best model on full training set
    best_ngram_counts, best_context_counts = build_ngram_model(train_set, best_n)
    best_probabilities = compute_probabilities(best_ngram_counts, best_context_counts)

    # Compute perplexity on the full test set for student model
    test_pp = perplexity(test_set, best_probabilities, best_n)
    print(f"Test set Perplexity for {best_n}-gram model: {test_pp}")  # Replace YYY.00000 with actual value

    # Generate JSON output for student model (first 100 predictions)
    generate_results_json(test_set, best_probabilities, best_n, "results_student_model.json")

    student_model = {
    "ngram_counts": best_ngram_counts,
    "context_counts": best_context_counts,
    "probabilities": best_probabilities,
    "n": best_n,
    "vocab": vocab  # Optional: include if needed for later inference
    }
    with open("student_model.pkl", "wb") as f:
        pickle.dump(student_model, f)
    print("Student model saved as 'student_model.pkl'.")

In [ ]:
def teacher_train():
    # Load dataset from cleaned corpus (e.g., 7561 methods)
    with open("/content/drive/Shareddrives/CSCI_420/data/new.txt", "r") as f:
        corpus = [line.strip() for line in f]

    # Training, Evaluation, and Testing on the Instructor-Provided Corpus
    with open("/content/drive/Shareddrives/CSCI_420/data/training.txt", "r") as f:
        instructor_corpus = [line.strip() for line in f]

    print(f"Total methods in instructor's corpus: {len(instructor_corpus)}")

    # Vocabulary Generation (using training + eval + test sets)
    vocab = generate_vocabulary(instructor_corpus)
    # print(vocab)
    print(f"Vocabulary size: {len(vocab)}")  # ZZZ number of code tokens

    random.shuffle(instructor_corpus)
    test_set = corpus[int(0.9 * len(corpus)):]
    instructor_train_set = instructor_corpus[:int(0.8 * len(instructor_corpus))]
    instructor_eval_set = instructor_corpus[int(0.8 * len(instructor_corpus)):int(0.9 * len(instructor_corpus))]

    best_instructor_n = None
    best_instructor_perplexity = float("inf")

    for n in [3, 5, 9]:
        print(f"Evaluating {n}-gram model on instructor's training data")
        ngram_counts, context_counts = build_ngram_model(instructor_train_set, n)
        probabilities = compute_probabilities(ngram_counts, context_counts)

        pp = perplexity(instructor_eval_set, probabilities, n)
        print(f"{n}-gram model Perplexity on Instructor's Eval set: {pp}")
        if pp < best_instructor_perplexity:
            best_instructor_perplexity = pp
            best_instructor_n = n

    print(f"Best N for instructor model: {best_instructor_n} (Perplexity: {best_instructor_perplexity})")
    # According to our report, n = 3 was selected for the instructor corpus as well

    # Train best instructor model
    best_instructor_ngram_counts, best_instructor_context_counts = build_ngram_model(instructor_train_set, best_instructor_n)
    best_instructor_probabilities = compute_probabilities(best_instructor_ngram_counts, best_instructor_context_counts)

    # Compute perplexity on test set for instructor model
    instructor_test_pp = perplexity(test_set, best_instructor_probabilities, best_instructor_n)
    print(f"Test set Perplexity for instructor's {best_instructor_n}-gram model: {instructor_test_pp}")

    # Generate JSON output for instructor model (first 100 predictions)
    generate_results_json(test_set, best_instructor_probabilities, best_instructor_n, "results_teacher_model.json")

    teacher_model = {
    "ngram_counts": best_instructor_ngram_counts,
    "context_counts": best_instructor_context_counts,
    "probabilities": best_instructor_probabilities,
    "n": best_instructor_n,
    "vocab": vocab  # Optional: include if needed for later inference
    }
    with open("teacher_model.pkl", "wb") as f:
        pickle.dump(teacher_model, f)
    print("Teacher model saved as 'teacher_model.pkl'.")

In [ ]:
def main():
    student_train()

if __name__ == "__main__":
    main()

Total methods in corpus: 7561
Vocabulary size: 22520
Train set: 6048 methods
Eval set: 756 methods
Test set: 757 methods
Evaluating 3-gram model on training data
Number of N-grams: 128314
3-gram model Perplexity on Eval set: 160.05503101771296
Evaluating 5-gram model on training data
Number of N-grams: 212233
5-gram model Perplexity on Eval set: 2220.84871797411
Evaluating 9-gram model on training data
Number of N-grams: 265173
9-gram model Perplexity on Eval set: 46245.58967958037
Selected best N: 3 (Perplexity: 160.05503101771296)
Number of N-grams: 128314
Test set Perplexity for 3-gram model: 168.11240212566014
Saved predictions to results_student_model.json
First prediction for method 0: ('equals', 0.026)
Student model saved as 'student_model.pkl'.


In [ ]:
def main():
    teacher_train()

if __name__ == "__main__":
    main()

Total methods in instructor's corpus: 100000
Vocabulary size: 86278
Evaluating 3-gram model on instructor's training data
Number of N-grams: 352518
3-gram model Perplexity on Instructor's Eval set: 26.999173669843625
Evaluating 5-gram model on instructor's training data
Number of N-grams: 646552
5-gram model Perplexity on Instructor's Eval set: 92.77275274502723
Evaluating 9-gram model on instructor's training data
Number of N-grams: 930439
9-gram model Perplexity on Instructor's Eval set: 1469.899841351272
Best N for instructor model: 3 (Perplexity: 26.999173669843625)
Number of N-grams: 352518
Test set Perplexity for instructor's 3-gram model: 5812.599899572303
Saved predictions to results_teacher_model.json
First prediction for method 0: ('accept', 0.229)
Teacher model saved as 'teacher_model.pkl'.


In [ ]:
from google.colab import files
files.download('results_student_model.json')
files.download('results_teacher_model.json')
files.download('student_model.pkl')
files.download('teacher_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>